# Unit 3. Transformer architectures for audio

Bu derste öncelikle transformatör modellerini ve bunların ses görevlerine nasıl uygulanabileceğini ele alacağız. Bu modellerin iç ayrıntılarını bilmenize gerek olmasa da, onları çalıştıran ana kavramları anlamak yararlıdır, bu yüzden burada hızlı bir tazeleme var. Transformatörleri derinlemesine incelemek için NLP Kursumuza göz atın.

## How does a transformer work?

Orijinal dönüştürücü modeli, yazılı metni bir dilden diğerine çevirmek için tasarlanmıştı. Mimarisi şu şekildeydi:

![](https://huggingface.co/datasets/huggingface-course/documentation-images/resolve/main/en/chapter1/transformers.svg)

Sol tarafta kodlayıcı, sağ tarafta ise kod çözücü yer almaktadır.

**Kodlayıcı** bir girdi, bu durumda bir dizi metin belirteci alır ve bunun bir **temsilini (özelliklerini) oluşturur**. **Modelin bu kısmı girdiden anlam çıkarmak için eğitilir**.

**Kod çözücü**, hedef diziyi oluşturmak için kodlayıcının temsilini (özellikler) diğer girdilerle (önceden tahmin edilen belirteçler) birlikte kullanır. Modelin bu kısmı **çıktılar üretmek için eğitilir**. Orijinal tasarımda, çıktı dizisi metin belirteçlerinden oluşuyordu.

**Yalnızca kodlayıcı** kısmını (**sınıflandırma** gibi girdinin anlaşılmasını gerektiren görevler için iyidir) veya **yalnızca kod çözücü** kısmını (**metin oluşturma** gibi görevler için iyidir) kullanan dönüştürücü tabanlı modeller de vardır. **Yalnızca kodlayıcı** modeline örnek olarak **BERT**; **yalnızca kod çözücü** modeline örnek olarak da **GPT2** verilebilir.

Transformatör modellerinin önemli bir özelliği, dikkat katmanları adı verilen özel katmanlarla inşa edilmeleridir. **Bu katmanlar modele girdi dizisindeki belirli unsurlara özel dikkat göstermesini ve özellik temsillerini hesaplarken diğerlerini göz ardı etmesini söyler**.

## Using transformers for audio

Bu kursta ele alacağımız ses modelleri genellikle yukarıda gösterildiği gibi standart bir dönüştürücü mimarisine sahiptir, ancak **metin yerine ses verilerine izin vermek için giriş veya çıkış tarafında küçük bir değişiklik yapılır**. Tüm bu modeller özünde dönüştürücü olduğundan, mimarilerinin çoğu ortak olacaktır ve **temel farklılıklar nasıl eğitildikleri** ve kullanıldıklarıdır.

![](https://huggingface.co/datasets/huggingface-course/audio-course-images/resolve/main/transformers_blocks.png)

Ses görevleri için, giriş ve/veya çıkış dizileri metin yerine ses olabilir:

- Otomatik konuşma tanıma (ASR): Giriş konuşmadır, çıkış metindir.

- Konuşma sentezi (TTS): Giriş metin, çıkış konuşmadır.

- Ses sınıflandırması: Girdi sestir, çıktı bir sınıf olasılığıdır - dizideki her öğe için bir tane veya tüm dizi için tek bir sınıf olasılığı.

- Ses dönüştürme veya konuşma geliştirme: Hem girdi hem de çıktı sestir.

Bir transformatör ile kullanılabilmesi için sesi işlemenin birkaç farklı yolu vardır. Ana husus, sesi ham haliyle - dalga formu olarak - kullanmak veya bunun yerine **spektrogram** olarak işlemektir.

## Model inputs

Bir ses modelinin girdisi metin ya da ses olabilir. Amaç, bu girdiyi dönüştürücü mimarisi tarafından işlenebilecek bir **gömme vektörüne** dönüştürmektir.

### Text input

Bir metinden konuşmaya modeli metni girdi olarak alır. Bu tıpkı orijinal dönüştürücü veya diğer NLP modelleri gibi çalışır: Girdi metni önce bir dizi metin belirteci verecek şekilde belirtecleştirilir. Bu dizi, belirteçleri 512 boyutlu vektörlere dönüştürmek için bir girdi gömme katmanına gönderilir. Bu gömme vektörleri daha sonra transformatör kodlayıcısına aktarılır.

### Waveform input

Bir otomatik konuşma tanıma modeli sesi girdi olarak alır. ASR için bir dönüştürücü kullanabilmek için öncelikle sesi bir şekilde bir dizi gömme vektörüne dönüştürmemiz gerekir.

**Wav2Vec2** ve **HuBERT** gibi modeller, **ses dalga biçimini doğrudan modelin girdisi olarak kullanır**. Ses verileriyle ilgili bölümde gördüğünüz gibi, bir dalga biçimi, her sayının belirli bir zamanda örneklenen genliği temsil ettiği tek boyutlu bir kayan noktalı sayı dizisidir. Bu ham dalga biçimi önce **sıfır ortalama** ve **birim varyansa** **normalleştirilir**, bu da ses örneklerinin farklı hacimlerde (genliklerde) standartlaştırılmasına yardımcı olur.

![](https://huggingface.co/datasets/huggingface-course/audio-course-images/resolve/main/wav2vec2-input.png)

Normalleştirmeden sonra, ses örnekleri dizisi, özellik kodlayıcı olarak bilinen küçük bir **konvolüsyonel sinir ağı** kullanılarak bir **gömme haline getirilir**. Bu ağdaki konvolüsyonel katmanların her biri, son konvolüsyonel katman her 25 ms'lik ses için gömme ile 512 boyutlu bir vektör çıkarana kadar, dizi uzunluğunu azaltmak için sesi alt örnekleyerek giriş dizisini işler. Giriş dizisi bu tür katıştırmaların bir dizisine dönüştürüldükten sonra, dönüştürücü verileri her zamanki gibi işleyecektir.

### Spectrogram input

Ham dalga biçimini girdi olarak kullanmanın bir dezavantajı, uzun dizi uzunluklarına sahip olma eğiliminde olmalarıdır. Örneğin, 16 kHz örnekleme hızında otuz saniyelik ses 30 * 16000 = 480000 uzunluğunda bir girdi verir. Daha uzun dizi uzunlukları, transformatör modelinde daha fazla hesaplama ve dolayısıyla daha yüksek bellek kullanımı gerektirir.

Bu nedenle, **ham ses dalga formları genellikle bir ses girişini temsil etmenin en verimli şekli değildir**. Bir **spektrogram kullanarak, aynı miktarda bilgiyi daha sıkıştırılmış bir biçimde elde ederiz**.

**Whisper gibi modeller önce dalga biçimini log-mel spektrogramına dönüştürür**. Whisper sesi her zaman 30 saniyelik bölümlere ayırır ve her bölüm için log-mel spektrogramı (80, 3000) şeklindedir; burada 80 mel kutusu sayısı ve 3000 dizi uzunluğudur. Log-mel spektrogramına dönüştürerek giriş verisi miktarını azalttık, ancak daha da önemlisi, bu ham dalga biçiminden çok daha kısa bir dizidir. Log-mel spektrogramı daha sonra küçük bir CNN tarafından her zamanki gibi dönüştürücüye giden bir gömme dizisine işlenir.

Her iki durumda da, hem dalga formu hem de spektrogram girdisi, dönüştürücünün önünde girdiyi gömülmelere dönüştüren küçük bir ağ vardır ve ardından dönüştürücü işini yapmak için devralır.

## Model outputs

Dönüştürücü mimarisi, çıktı gömülmeleri olarak da bilinen **bir dizi gizli durum vektörü çıkarır**. Amacımız bu vektörleri bir metin veya ses çıktısına dönüştürmektir.

### Text output

Otomatik bir konuşma tanıma modelinin amacı, bir dizi metin simgesini tahmin etmektir. Bu, bir dil modelleme başlığı - tipik olarak tek bir doğrusal katman - ve ardından dönüştürücünün çıktısının üzerine bir softmax eklenerek yapılır. Bu, kelime dağarcığındaki metin belirteçleri üzerindeki olasılıkları tahmin eder.

### Spectrogram output

Metinden konuşmaya (TTS) modeli gibi ses üreten modeller için, bir ses dizisi üretebilen katmanlar eklememiz gerekecektir. Bir spektrogram oluşturmak ve ardından bu spektrogramı bir dalga formuna dönüştürmek için vocoder olarak bilinen ek bir sinir ağı kullanmak çok yaygındır.

Örneğin SpeechT5 TTS modelinde, transformatör ağından elde edilen çıktı 768 elemanlı vektörlerden oluşan bir dizidir. Doğrusal bir katman bu diziyi log-mel spektrogramına yansıtır. Ek doğrusal ve evrişimsel katmanlardan oluşan ve post-net olarak adlandırılan bir katman, gürültüyü azaltarak spektrogramı rafine eder. Vokoder daha sonra nihai ses dalga formunu oluşturur.

![](https://huggingface.co/datasets/huggingface-course/audio-course-images/resolve/main/speecht5.png)

### Waveform output

Modellerin bir ara adım olarak spektrogram yerine doğrudan bir dalga formu çıkarması da mümkündür, ancak şu anda Transformers'da bunu yapan herhangi bir modelimiz yok.

## Conclusion

Özetle: Çoğu ses transformatörü modeli farklı olmaktan çok birbirine benzer - hepsi aynı transformatör mimarisi ve dikkat katmanları üzerine inşa edilmiştir, ancak bazı modeller transformatörün yalnızca kodlayıcı kısmını kullanırken diğerleri hem kodlayıcı hem de kod çözücü kullanır.

Ses verilerinin transformatör modellerine nasıl girip çıktığını da gördünüz. ASR, TTS ve benzeri farklı ses görevlerini gerçekleştirmek için, dönüştürücü omurgası aynı kalırken, girdileri katıştırmalara ön işleme tabi tutan katmanları değiştirebilir ve tahmin edilen katıştırmaları çıktılara son işleme tabi tutan katmanları değiştirebiliriz.

Daha sonra, bu modellerin otomatik konuşma tanıma için eğitilebileceği birkaç farklı yola bakalım.